In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from googletrans import Translator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
from sklearn.model_selection import KFold
from imblearn.under_sampling import InstanceHardnessThreshold
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import openpyxl
from tqdm import tqdm  
from googletrans import Translator
# Translation
from tqdm import tqdm  
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, OneSidedSelection, InstanceHardnessThreshold
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import KFold
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from bidi.algorithm import get_display

In [ ]:
# Load Data
wb = openpyxl.load_workbook('E:/Research_Work/Bangla_CyberBulling_2024/Code & dataset/dataset/Final_Dataset_94k_Bangla_CyberBulling_Combine_Dataset_94k.xlsx')
ws = wb['Sheet1']
data_rows = []
for row in ws['A1':'B94001']:
    data_cols = [cell.value for cell in row]
    data_rows.append(data_cols)
df = pd.DataFrame(data_rows)
header = df.iloc[0]
df.columns = [header]
df = df.iloc[1:, :]

In [ ]:
class_counts = df['label'].value_counts()
print("Class Counts:\n", class_counts)

In [ ]:
all_comments = ' '.join(df['comment'].apply(lambda x: ' '.join(map(str, x))).astype(str))

rgx = r"[\u0980-\u09FF]+"
wordcloud = WordCloud(font_path='E:/Research_Work/Bangla_CyberBulling_2024/Code & dataset/Dataset/Font/CHANO___.ttf',regexp=rgx, width=2000, height=1300, background_color='white').generate(all_comments)

plt.figure(figsize=(30, 15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Data Exploration
print(df.head())
print(df.iloc[:, -1].value_counts())
print(df.shape)
print(df.isna().sum())

# Handle Missing Values
mode = df.iloc[:, -2].value_counts().index[0]
df.iloc[:, -2].fillna(mode, inplace=True)
print(df.isna().sum())
print(df.head())

In [ ]:
# Labeling and Encoding
list(df.iloc[:, -1].value_counts().index)
df['nlabel'] = df.label.replace(['not bully', 'troll', 'sexual', 'religious', 'threat'], [0, 1, 2, 3, 4])
df['noutput'] = df.iloc[:, -1].replace(['bully', 'normal'], [1, 0])
print(df.head())
print(df.head())

In [ ]:
translator = Translator()
output = []

for i in tqdm(range(len(df))):
    if df.label.values[i] == 'not bully':
        output.append('Normal')
    else:
        output.append('Bully')

df['output'] = output
df['noutput'] = df.iloc[:, -1].replace(['Bully', 'Normal'], [1, 0])


In [ ]:
df['comment']

In [ ]:
df.to_csv('v1.csv')

In [ ]:
print(stopwords.fileids())
sw = stopwords.words('bengali')

new_stopwords = stopwords.words('english')
new_stopwords.append('SampleWord')

df.to_csv('df_english1000.csv')

nltk.download('wordnet')
nltk.download('omw-1.4')

wordnet_lemmatizer = WordNetLemmatizer()

M = len(df)
for k in tqdm(range(M)):
    text = df.iloc[:, 0].values[k]
    txt = re.sub(r'http\S+', '', text)
    txt = re.sub(r'[!@#$%^&*?><,./\-+`~|:);(❤}{]', '', txt)

    text_tokens = word_tokenize(txt)

    def detect_benglish(text_tokens):
        benglish = False
        for j in range(len(text_tokens)):
            for i in range(len(text_tokens[j])):
                detector = Translator()
                txt = text_tokens[j]
                if i < len(txt) - 1:
                    if detector.detect(txt[i]).lang == 'bn' and detector.detect(txt[i + 1]).lang == 'en' and txt[
                        i].isalpha() == True and txt[i + 1].isalpha() == True:
                        benglish = True
        return benglish

    def handle_benglish(text_tokens):
        for j in range(len(text_tokens)):
            s = []
            for i in range(len(text_tokens[j])):
                detector = Translator()
                txt = text_tokens[j]
                if i < len(txt) - 1:
                    if detector.detect(txt[i]).lang == 'bn' and detector.detect(txt[i + 1]).lang == 'en' and txt[
                        i].isalpha() == True and txt[i + 1].isalpha() == True:
                        s.append(i)
                    if txt[i] == '|':
                        s.append(i)
            for i in range(len(s)):
                txt = txt.replace(txt[s[i]], txt[s[i]] + " ")
            text_tokens[j] = txt
        tstring = str(' '.join(text_tokens)).lower()
        text_tokens = word_tokenize(tstring)
        return text_tokens

    remove_sw = [word for word in text_tokens if not word in sw]
    un_items = np.unique(remove_sw)
    r_sw = [wordnet_lemmatizer.lemmatize(w) for w in un_items]
    bn_tokens = []

    def tanslate_bengali(r_sw):
        for i in range(len(r_sw)):
            bn_tokens.append(translator.translate(r_sw[i], dest='bn').text)
        return bn_tokens

    bn_token = r_sw
    df.iloc[:, 0].values[k] = ' '.join(bn_token)

df.iloc[:, -1].value_counts()
df.iloc[:, 0]

df[:].iloc[:, 0].shape


In [ ]:
df

In [ ]:
# Text Vectorization (Count)
count_vectorizer = CountVectorizer(max_features=20000, tokenizer=word_tokenize, stop_words=stopwords.words('english'))
iX_count = count_vectorizer.fit_transform(df[:].iloc[:, 0].values).toarray()

In [ ]:
tfidf_vectorizer = TfidfTransformer()
tf_transformer = TfidfTransformer(use_idf=True).fit(iX_count)
iX_tfidf = tf_transformer.transform(iX_count)
iX_tfidf.shape

In [ ]:
by = df.iloc[:, -3].values
by.shape

my = df.iloc[:, -1].values

np.unique(by)

In [ ]:
# Imbalanced Data Handling
cc = InstanceHardnessThreshold(random_state=10, estimator=LogisticRegression())
X, y = cc.fit_resample(iX_tfidf, by)
X.shape, y.shape

In [ ]:
# Cross-Validation
kf = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
print('complete')
print('train:', X_train.shape, y_train.shape, 'test:', X_test.shape, y_test.shape)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 8: Model Building (Logistic Regression)
def build_models_logistic_regression():
    print('Building Logistic Regression model...')
    logistic_regression = LogisticRegression()
    logistic_regression.fit(X_train, y_train)
    print('Logistic Regression model built successfully.')
    return logistic_regression

logistic_regression = build_models_logistic_regression()

# Part 9: Model Evaluation (Logistic Regression)
def evaluate_model(logistic_regression, X_test, y_test):
    y_pred = logistic_regression.predict(X_test)
    
    # Calculate evaluation metrics
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Print the metrics
    print(f'Accuracy: {acc*100:.2f}%')
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    return acc, precision, recall, f1

accuracy_logistic_regression, precision_logistic_regression, recall_logistic_regression, f1_logistic_regression = evaluate_model(logistic_regression, X_test, y_test)

# Part 10: Confusion Matrix Analysis (Logistic Regression)
# Part 10: Confusion Matrix Analysis (Logistic Regression)
def confusion_matrix_analysis(logistic_regression, X_test, y_test):
    y_pred = logistic_regression.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (Logistic Regression)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

confusion_matrix_analysis(logistic_regression, X_test, y_test)


# Part 11: ROC Curve (Logistic Regression)
def plot_roc_curve(logistic_regression, X_test, y_test, class_names):
    y_probs = logistic_regression.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (LR)')
    plt.legend()
    plt.grid(True)
    plt.show()

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
plot_roc_curve(logistic_regression, X_test, y_test, class_names)

In [ ]:
print('Coefficients:', logistic_regression.coef_)

# XGBoost

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 8: Model Building (XGBoost)
def build_models_xgboost():
    print('Building XGBoost model...')
    xgboost_model = XGBClassifier()
    xgboost_model.fit(X_train, y_train)
    print('XGBoost model built successfully.')
    return xgboost_model

xgboost_model = build_models_xgboost()

# Part 9: Model Evaluation (XGBoost)
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    # Calculate precision, recall, and F1 score
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    # Print the metrics
    print(f'Accuracy: {acc*100:.2f}%')
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')

    return acc, precision, recall, f1

accuracy_xgboost, precision_xgb, recall_xgb, f1_xgb = evaluate_model(xgboost_model, X_test, y_test)

# Part 10: Confusion Matrix Analysis (XGBoost)
def confusion_matrix_analysis(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (xgboost_model)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

confusion_matrix_analysis(xgboost_model, X_test, y_test)

# Part 11: ROC Curve (XGBoost)
def plot_roc_curve(model, X_test, y_test, class_names):
    y_probs = model.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (XGBoost)')
    plt.legend()
    plt.grid(True)
    plt.show()

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
plot_roc_curve(xgboost_model, X_test, y_test, class_names)

# AdaBoost

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 16: Model Building (AdaBoost)
def build_models_adaboost(X_train, y_train):
    global adaboost_model
    print('Building AdaBoost model...')
    base_model = DecisionTreeClassifier(max_depth=1)
    adaboost_model = AdaBoostClassifier(base_model, n_estimators=50)
    adaboost_model.fit(X_train, y_train)
    print('AdaBoost model built successfully.')
    return adaboost_model

adaboost_model = build_models_adaboost(X_train, y_train)

# Part 17: Model Evaluation (AdaBoost)
def evaluate_model_adaboost(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc*100:.2f}%')
    
    # Additional metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    return acc

accuracy_adaboost = evaluate_model_adaboost(adaboost_model, X_test, y_test)

# Part 18: Confusion Matrix Analysis (AdaBoost)
def confusion_matrix_analysis_adaboost(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (AdaBoost)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

confusion_matrix_analysis_adaboost(adaboost_model, X_test, y_test)


# Part 19: ROC Curve (AdaBoost)
def plot_roc_curve_adaboost(model, X_test, y_test, class_names):
    y_probs = model.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (AdaBoost)')
    plt.legend()
    plt.grid(True)
    plt.show()

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
plot_roc_curve_adaboost(adaboost_model, X_test, y_test, class_names)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 23: Model Building (Random Forest)
def build_models_random_forest(X_train, y_train):
    global random_forest_model
    print('Building Random Forest model...')
    random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
    random_forest_model.fit(X_train, y_train)
    print('Random Forest model built successfully.')
    return random_forest_model

random_forest_model = build_models_random_forest(X_train, y_train)

# Part 24: Model Evaluation (Random Forest)
def evaluate_model_random_forest(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc*100:.2f}%')
    
    # Additional metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    return acc

accuracy_random_forest = evaluate_model_random_forest(random_forest_model, X_test, y_test)

# Part 25: Confusion Matrix Analysis (Random Forest)
def confusion_matrix_analysis_random_forest(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (Random Forest)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

confusion_matrix_analysis_random_forest(random_forest_model, X_test, y_test)

# Part 19: ROC Curve (AdaBoost)
def roc_curve_analysis_random_forest(random_forest_model, X_test, y_test, class_names):
    y_probs = random_forest_model.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (RF)')
    plt.legend()
    plt.grid(True)
    plt.show()

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
roc_curve_analysis_random_forest(random_forest_model, X_test, y_test, class_names)

# K-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 27: Model Building (K-NN)
def build_models_knn(X_train, y_train):
    global knn_model
    print('Building K-NN model...')
    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(X_train, y_train)
    print('K-NN model built successfully.')
    return knn_model

knn_model = build_models_knn(X_train, y_train)

# Part 28: Model Evaluation (K-NN)
def evaluate_model_knn(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc*100:.2f}%')
    
    # Additional metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    return acc

accuracy_knn = evaluate_model_knn(knn_model, X_test, y_test)

# Part 29: Confusion Matrix Analysis (K-NN)
def confusion_matrix_analysis_knn(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (K-NN)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

confusion_matrix_analysis_knn(knn_model, X_test, y_test)

# Part 19: ROC Curve (AdaBoost)
def roc_curve_analysis_knn_model(knn_model, X_test, y_test, class_names):
    y_probs = knn_model.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (knn)')
    plt.legend()
    plt.grid(True)
    plt.show()

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
roc_curve_analysis_knn_model(knn_model, X_test, y_test, class_names)

# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 31: Model Building (Bagging with Decision Tree)
def build_models_bagging(X_train, y_train):
    global bagging_model
    print('Building Bagging model...')
    base_estimator = DecisionTreeClassifier(max_depth=5)
    bagging_model = BaggingClassifier(base_estimator, n_estimators=10, random_state=42)
    bagging_model.fit(X_train, y_train)
    print('Bagging model built successfully.')
    return bagging_model

bagging_model = build_models_bagging(X_train, y_train)

# Part 32: Model Evaluation (Bagging)
def evaluate_model_bagging(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc*100:.2f}%')
    
    # Additional metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    return acc

accuracy_bagging = evaluate_model_bagging(bagging_model, X_test, y_test)

# Part 33: Confusion Matrix Analysis (Bagging)
def confusion_matrix_analysis_bagging(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (Bagging)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()
    
confusion_matrix_analysis_bagging(bagging_model, X_test, y_test)

# Part 19: ROC Curve (AdaBoost)
def roc_curve_analysis_bagging_model(bagging_model, X_test, y_test, class_names):
    y_probs = bagging_model.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Bagging)')
    plt.legend()
    plt.grid(True)
    plt.show()

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
roc_curve_analysis_bagging_model(bagging_model, X_test, y_test, class_names)

# Multi-Layer Perceptron (MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 43: Model Building (MLP)
def build_model_mlp(X_train, y_train):
    global mlp_model
    print('Building MLP model...')
    mlp_model = MLPClassifier()
    mlp_model.fit(X_train, y_train)
    print('MLP model built successfully.')
    return mlp_model

mlp_model = build_model_mlp(X_train, y_train)

# Part 44: Model Evaluation (MLP)
def evaluate_model_mlp(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc*100:.2f}%')
    
    # Additional metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    return acc

accuracy_mlp = evaluate_model_mlp(mlp_model, X_test, y_test)

# Part 45: Confusion Matrix Analysis (MLP)
def confusion_matrix_analysis_mlp(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (MLP)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

confusion_matrix_analysis_mlp(mlp_model, X_test, y_test)

# Part 46: ROC Curve (MLP)
def roc_curve_analysis_mlp_model(mlp_model, X_test, y_test, class_names):
    y_probs = mlp_model.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (mlp)')
    plt.legend()
    plt.grid(True)
    plt.show()

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
roc_curve_analysis_mlp_model(mlp_model, X_test, y_test, class_names)



# SGDClassifier__ Stochastic Gradient Descent Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 47: Model Building (SGD Classifier)
def build_model_sgd(X_train, y_train):
    global sgd_model
    print('Building SGD Classifier model...')
    sgd_model = SGDClassifier()
    sgd_model.fit(X_train, y_train)
    print('SGD Classifier model built successfully.')
    return sgd_model

sgd_model = build_model_sgd(X_train, y_train)

# Part 48: Model Evaluation (SGD Classifier)
def evaluate_model_sgd(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc*100:.2f}%')
    
    # Additional metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    return acc

accuracy_sgd = evaluate_model_sgd(sgd_model, X_test, y_test)

# Part 49: Confusion Matrix Analysis (SGD Classifier)
def confusion_matrix_analysis_sgd(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (SGD Classifier)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

confusion_matrix_analysis_sgd(sgd_model, X_test, y_test)


# Part 50: ROC Curve (SGD Classifier)
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

def roc_curve_analysis_sgd_model(sgd_model, X_test, y_test, class_names):
    # Fit the SGDClassifier first
    sgd_model.fit(X_train, y_train)
    
    # Calibrate the classifier to get probability estimates
    calibrated_sgd = CalibratedClassifierCV(sgd_model, method='sigmoid', cv='prefit')
    calibrated_sgd.fit(X_train, y_train)
    
    # Get probability estimates
    y_probs = calibrated_sgd.predict_proba(X_test)

    plt.figure(figsize=(6, 4))
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (SGD)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Assuming you have already defined and trained your SGDClassifier
sgd_model = SGDClassifier(random_state=42)  # Remove loss='log'

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']
roc_curve_analysis_sgd_model(sgd_model, X_test, y_test, class_names)


In [ ]:
# SGD+LR+MLP (Stacking)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import is_classifier

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code
# Part 51: Model Building (Stacking Classifier: SGD, MLP, RandomForest)
def build_stacking_classifier(X_train, y_train):
    global stacking_model
    print('Building Stacking Classifier model...')
    # Creating instances of the base classifiers
    sgd_classifier = SGDClassifier()
    mlp_classifier = MLPClassifier()
    rf_classifier = RandomForestClassifier()
    # Check if classifiers support probability estimates
    classifiers_with_proba = [(name, clf) for name, clf in [('sgd', sgd_classifier), ('mlp', mlp_classifier), ('rf', rf_classifier)] if is_classifier(clf) and hasattr(clf, "predict_proba")]
    if not classifiers_with_proba:
        raise ValueError("None of the base classifiers support probability estimates. Stacking requires probability estimates.")

    # Creating the Stacking Classifier
    stacking_model = StackingClassifier(classifiers_with_proba, final_estimator=SGDClassifier(), stack_method='auto', n_jobs=-1)
    
    # Fitting the model
    stacking_model.fit(X_train, y_train)
    print('Stacking Classifier model built successfully.')
    return stacking_model
# Building the Stacking Classifier (assuming X_train, X_test, y_train, y_test are defined)
stacking_model = build_stacking_classifier(X_train, y_train)

# Part 52: Model Evaluation (Stacking Classifier)
def evaluate_stacking_classifier(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc*100:.2f}%')
    
    # Additional metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    confusion_matrix_analysis(model, X_test, y_test)
    
    return acc

# Part 53: Confusion Matrix Analysis (Stacking Classifier)
def confusion_matrix_analysis(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (Stacking Classifier)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

# Evaluating the Stacking Classifier
accuracy_stacking = evaluate_stacking_classifier(stacking_model, X_test, y_test)

In [ ]:
# Part 54: ROC Curve (LightGBM with Default Parameters)
def roc_curve_analysis_stacking_model(stacking_model, X_test, y_test, class_names):
    y_probs = stacking_model.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (stacking)')
    plt.legend()
    plt.grid(True)
    plt.show() 

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
roc_curve_analysis_stacking_model(stacking_model, X_test, y_test, class_names)

In [ ]:
from sklearn.metrics import classification_report

# Assuming you have defined and trained your stacking_model
stacking_model.fit(X_train, y_train)  # Make sure to fit the model before using it

# Part 53: Classification Report Analysis (Stacking Classifier)
def classification_report_analysis(model, X_test, y_test):
    y_pred = model.predict(X_test)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names

    # Classification Report
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=class_names))

# Evaluating the Stacking Classifier
accuracy_stacking = evaluate_stacking_classifier(stacking_model, X_test, y_test)
classification_report_analysis(stacking_model, X_test, y_test)


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import is_classifier

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 51: Model Building (Voting Classifier: SGD, Logistic Regression, MLP)
def build_voting_classifier(X_train, y_train):
    global voting_model
    print('Building Voting Classifier model...')
    
    # Creating instances of the classifiers
    sgd_classifier = SGDClassifier()
    lr_classifier = LogisticRegression()
    mlp_classifier = MLPClassifier()

    # Check if classifiers support probability estimates
    classifiers_with_proba = [(name, clf) for name, clf in [('sgd', sgd_classifier), ('lr', lr_classifier), ('mlp', mlp_classifier)] if is_classifier(clf) and hasattr(clf, "predict_proba")]

    if not classifiers_with_proba:
        raise ValueError("None of the base classifiers support probability estimates. Soft voting requires probability estimates.")

    # Creating the Voting Classifier
    voting_model = VotingClassifier(classifiers_with_proba, voting='soft')  # 'hard' for majority voting
    
    # Fitting the model
    voting_model.fit(X_train, y_train)
    
    print('Voting Classifier model built successfully.')
    return voting_model

# Building the Voting Classifier (assuming X_train, X_test, y_train, y_test are defined)
voting_model = build_voting_classifier(X_train, y_train)

# Part 52: Model Evaluation (Voting Classifier)
def evaluate_voting_classifier(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc*100:.2f}%')
    
    # Additional metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    confusion_matrix_analysis(model, X_test, y_test)
    
    return acc

# Part 53: Confusion Matrix Analysis (Voting Classifier)
def confusion_matrix_analysis(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['not bully', 'troll', 'sexual', 'religious', 'threat']  # Update class names
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10)

    plt.title('Confusion Matrix (Voting Classifier)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

# Evaluating the Voting Classifier
accuracy_voting = evaluate_voting_classifier(voting_model, X_test, y_test)

# Part 11: ROC Curve (LightGBM with Default Parameters)
from sklearn.metrics import roc_curve, auc
def roc_curve_analysis_voting_model(voting_model, X_test, y_test, class_names):
    y_probs = voting_model.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (voting)')
    plt.legend()
    plt.grid(True)
    plt.show()

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
roc_curve_analysis_voting_model(voting_model, X_test, y_test, class_names)

In [ ]:
# Part 11: ROC Curve (LightGBM with Default Parameters)
from sklearn.metrics import roc_curve, auc
def roc_curve_analysis_voting_model(voting_model, X_test, y_test, class_names):
    y_probs = voting_model.predict_proba(X_test)

    plt.figure(figsize=(6, 4))  # Adjusted figure size
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=i)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')  # Adjusted class name format

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (voting)')
    plt.legend()
    plt.grid(True)
    plt.show()

class_names = ['Not Bully', 'Troll', 'Sexual', 'Religious', 'Threat']  # Adjusted class names format
roc_curve_analysis_voting_model(voting_model, X_test, y_test, class_names)